In [3]:
from pathlib import Path
"Module" in str(Path.cwd())

True

In [4]:
from XQuant import Tools
Tools.get_newest_file("ResConSecTarpriScore")

'E:\\Share\\Stk_Data\\dataFile\\ResConSecTarpriScore\\ResConSecTarpriScore_Y2023.h5'

In [5]:
import h5py

In [40]:
data = h5py.File('E:\\Share\\Stk_Data\\dataFile\\ResConSecTarpriScore\\ResConSecTarpriScore_Y2023.h5', 'r')

In [15]:
data["a"].keys()

<KeysViewHDF5 ['axis0', 'axis1', 'block0_items', 'block0_values', 'block1_items', 'block1_values', 'block2_items', 'block2_values']>

In [67]:
"S" in data["a"]["axis0"].dtype

TypeError: argument of type 'numpy.dtype[bytes_]' is not iterable

In [54]:
data["a"]["axis0"]

<HDF5 dataset "axis0": shape (12,), type "|S16">

In [63]:
data["a"]["axis1"].dtype == "int64"

True

In [49]:
data["a"]["axis0"][:][0].decode("utf-8") == "secCode"

AttributeError: 'numpy.ndarray' object has no attribute 'decode'

In [38]:
import pandas as pd
data = pd.HDFStore('E:\\Share\\Stk_Data\\dataFile\\ResConSecTarpriScore\\ResConSecTarpriScore_Y2023.h5', mode='r')

In [69]:
data["a"].keys()

<KeysViewHDF5 ['axis0', 'axis1', 'block0_items', 'block0_values', 'block1_items', 'block1_values', 'block2_items', 'block2_values']>

In [36]:
data["a"]["block1_items"][:]

array([b'conTarPriceType', b'conScoreType'], dtype='|S15')

In [43]:
data["a"]["block0_items"][:]

array([b'conTarPrice', b'conScore', b'closePrice', b'conTarReturn'],
      dtype='|S12')

In [35]:
data["a"]["block2_items"][:]

array([b'secCode', b'secName', b'repForeDate', b'tarPriceLastDate',
       b'scoreLastDate', b'updateTime'], dtype='|S16')